# Retrieval Augmented Generation (RAG) Techniques Applications

Os LLMs (Large Language Models) podem ser adaptados para realizar uma variedade de tarefas comuns, como análise de sentimentos e reconhecimento de entidades, sem a necessidade de conhecimento prévio adicional. No entanto, para tarefas mais complexas e que demandam mais conhecimento, como garantir consistência factual e evitar erros de "alucinação", os pesquisadores desenvolveram o método da Geração Aprimorada por Recuperação, a.k.a (`RAG`).

Essa abordagem combina um componente de recuperação de informações com um modelo gerador de texto, permitindo que o sistema acesse fontes externas de conhecimento para completar suas tarefas. Dessa forma, é interessante notar este tipo de método se torna capaz de lidar com informações que evoluem ao longo do tempo, sem necessidade de um novo ajuste-fino (que é computacionalmente caro).

Embora os LLMs tenham demonstrado habilidades poderosas, como impulsionar casos de uso avançados, eles ainda enfrentam desafios como inconsistências de fatos e erros de interpretação. Ao integrar **RAG** aos LLMs, é possível enriquecer suas capacidades e melhorar sua confiabilidade, permitindo a outputs (respostas) mais precisos e relevantes.

**Sobre este projeto:**

**RAG** tem se tornado uma técnica cada vez mais promissora, pois não é a toa que vemos na literatura artigos sobre aplicação de **RAG** para mitigação das alucinações, comparações com ajuste-fino e seus benefícios, evolução para ideias de implementação cada vez mais avançadas (e eficazes) e frameworks especializados para suportar aplicações com RAG.

Por isso, neste lab, venho com o intuito de ser capaz de implementar e apresentar algumas das técnicas muito interessantes de **RAG**.

<img src="img/RAG.png">

## Outline

0. [Packages and Keys;](#packages-and-keys)
1. [Production-Ready RAG Applications;](#production-rag)
2. [Naive RAG;](#naive-rag)
3. [Some Advanced Techniques;](#advanced-techniques)
4. [More other applications;](#other-apps)


<div id='packages-and-keys'></div>

## 0. Packages and Keys

Before we start building our applications, you (we) need to install some Python libraries. Here's a brief overview of what each library does:

- **Langchain**: This is a library for GenAI. We'll use it to chain together different language models and components for our chatbot.
- **OpenAI**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
- **Pinecone-client**: This is the official Pinecone Python client. We'll use it to interact with the Pinecone API and store our chatbot's knowledge base in a vector database.
- **Llama Index**: is an incredibly powerful tool for enhancing the capabilities of Large Language Models with your own data. Its array of data connectors, advanced query interfaces, and flexible integration make it a vital component in the development of applications with LLMs.

Após feito isso, vamos importar as dependências necessárias.

In [31]:
import os
import numpy as np
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()

True

Now, read (yours) the keys:

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")       # YOUR API KEY
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")   # YOUR API KEY
PINECONE_ENV_KEY = os.getenv("PINECONE_API_ENV")   # YOUR ENV

<div id='production-rag'></div>

## 1. Production-Ready RAG Applications
_Este tópico é baseado na apresentação de Jerry Liu, Co-Founder/CEO do LlamaIndex, no AI Engineer Summit._ 

É surpreendente o poder da IA Generativa. Hoje, pode-se observar uma gama de casos de uso incríveis com grande relevância para os LLM tais como: 

- Document Processing
- Tagging & Extraction
- Knowledge Search & Question Answering
- Conversational Agents
- Workflow Automation

Nesse sentido, também sabemos que grandes quantidade de dados são geradas todos os dias, o tempo todo, e tendo isso em vista, a fim de ensinar os LLMs a entender dados novos, temos em mãos duas abordagens capazes de suprir esta demanda: **retrieval augmented generation (RAG)**, que envolve adicionar contexto (por um Vector Database) ao prompt do modelo a partir de uma fonte de dados, e o **fine-tuning**, que atualiza os pesos da rede para incorporar conhecimento.

No entanto, como estamos focando em `RAG` no momento, vamos tratar um pouco agora sobre a construção de um sistema de perguntas e respostas. Este processo envolve duas etapas principais: **ingestão de dados** e **consulta de dados**, que inclui recuperação e síntese. 

<img src="img/rag-stack.png">

Embora esses componentes sejam fundamentais, e seja este o sistema que mais tem surgido atualmente, os desenvolvedores talvez começam a perceber que isto não é suficiente e passam a enfrentar desafios que impedem a produção eficaz dessas aplicações, como 
1. Baixa precisão e chunks irrelevantes, que podem causar halucinações;
2. Baixa recuperação, que não recupera todos os pedaços relevantes, implicando em contexto incompleto;
3. Informações desatualizadas, onde dados antigos tornam o LLM impreciso e incapaz de raciocinar.

Para superar esses desafios, Jerry sugere melhorias em toda a pipeline, desde armazenar informações adicionais e otimizar a pipeline de dados até explorar métodos avançados de recuperação e síntese, além da importância de ser capaz medir o desempenho de um sistema RAG (que podemos entrar em detalhes em outro momento). 

Nesse sentido, é isso que busco fazer aqui. Mas antes de buscarmos essas melhorias, vamos começar com um caso básico, isto é, o `Naive RAG`.

<div id='naive-rag'></div> 

## 2. Naive RAG

### 2.1. Introdução e entendendo o Naive-RAG

Atualmente, eu faço pesquisa (uma iniciação tecnológica) que trata sobre um _estudo comparativo de modelos de sistema de recomendação baseados em aprendizado por reforço_. Durante meus estudos, estive buscando entender uma certa implementação aberta acerca do artigo _Deep Reinforcement Learning based Recommendation with Explicit User-Item Interactions Modeling_. Perguntei ao ChatGPT se ele conhecia o artigo e se de alguma forma ele iria conseguir me ajudar. Ele felizmente conhecia, no entanto, apenas conhecia sua existência e do que se tratava o artigo, mas não sobre detalhes internos. Então pensei: "por que não usar **RAG** para ele ser capaz de explicar algumas das minhas dúvidas?". Logo, aqui mostrarei a aplicação de uma arquitetura básica de RAG
(`Naive RAG`) para este caso de uso específico.

Antes de começarmos, é interessante entender um pouco como funciona o esquema/arquitetura do Naive Rag. Segue abaixo.
<img src="img/naive-rag.png">

Resumidamente, o que fazemos é o seguinte: você **divide seus textos** (de seus documentos) em pedaços, depois **incorpora** esses pedaços em vetores com algum **Transformer Encoder model** ou **Embedding model**, coloca todos esses vetores em um **Vector Database** e, finalmente, **cria um prompt** para um LLM que diz ao modelo para **responder a consulta do usuário**, considerando o **contexto** que encontramos na etapa de busca.

No tempo de execução, **vetorizamos a consulta do usuário** com o mesmo Encoder model ou Embedding model e, em seguida, executamos a **pesquisa desse vetor de consulta no Vector Database**, encontramos os **k principais resultados**, recuperamos os pedaços de texto correspondentes de nosso banco de dados e os **alimentamos** no prompt do LLM **como contexto**.

### 2.2. Prototipagem
Como o objetivo aqui é trazer um conteúdo bem consolidado (e espero trazer, pelo menos) e apesar das discussões sobre quando usar **LangChain** ou **Llama Index** (e eu particularmente achar que talvez Llama Index seja mais apropriado), acredito que usar ambos será definitivamente divertido (embora mais caro rs). 

#### 2.2.1. Com LangChain

#### Fast overview
Inicialmente, vamos criar um chatbot simples sem qualquer tipo de contexto inserido, inicializando uma instância de `ChatOpenAI` com nossa chave da API da OpenAI e utilizando o modelo `gpt-3.5-turbo`.

In [3]:
chat = ChatOpenAI(
    api_key = OPENAI_API_KEY,
    model = "gpt-3.5-turbo-0125"
)

Chats com modelos de chat da OpenAI como o `gpt-3.5-turbo` e o `gpt-4-0125-preview` geralmente são estruturados da seguinte forma: 

```
System: You are a helpful assistant.

User: Hi Chat, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand about reinforcement learning theory.

Assistant:
```

O final `"Assistant:"` sem uma resposta é o que levaria o modelo a continuar a conversa. No caso do endpoint oficial do OpenAI `ChatCompletion`, eles seriam passados como:

```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi Chat, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand about reinforcement learning theory."}
]
```

No entanto, quando utilizamos o LangChain, há uma leve diferença que se dá por:

In [4]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi Chat, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand about reinforcement learning theory.")
]

No fim, esse formato vai se assemelhar bastante com o contexto oficial. Note que em termos de analogia, quando a nossa `role` era **user**, para o LangChain temos um objeto de **HumanMessage**; quando a nossa `role` era **assistant**, para o LangChain temos um objeto de **AIMessage** e, por fim, quando a nossa `role` era **system**, temos para o LangChain um objeto de **SystemMessage**.

Agora, para sermos capazes de gerar uma resposta, vamos passar estas mensagens para a instância do `ChatOpenAI` que criamos.

In [5]:
response = chat(messages)
response

C:\Users\samue\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="Reinforcement learning is a type of machine learning that involves an agent learning how to make decisions by interacting with an environment. The agent receives feedback in the form of rewards or penalties based on its actions, and uses this feedback to learn the best strategies for achieving its goals.\n\nIn reinforcement learning, the agent's goal is typically to maximize its cumulative reward over time. Through trial and error, the agent learns which actions lead to the most favorable outcomes and adjusts its behavior accordingly.\n\nSome key components of reinforcement learning include:\n\n1. Agent: The entity that learns and makes decisions within the environment.\n2. Environment: The external system with which the agent interacts.\n3. State: A representation of the current situation of the agent within the environment.\n4. Action: The choices available to the agent at any given state.\n5. Reward: The feedback signal that the agent receives after taking an acti

Note que obtemos uma mensagem do tipo AI Message. Se quisermos dar uma olhada mais clara na resposta podemos fazer:

In [6]:
print(response.content) # tomando o conteúdo da resposta

Reinforcement learning is a type of machine learning that involves an agent learning how to make decisions by interacting with an environment. The agent receives feedback in the form of rewards or penalties based on its actions, and uses this feedback to learn the best strategies for achieving its goals.

In reinforcement learning, the agent's goal is typically to maximize its cumulative reward over time. Through trial and error, the agent learns which actions lead to the most favorable outcomes and adjusts its behavior accordingly.

Some key components of reinforcement learning include:

1. Agent: The entity that learns and makes decisions within the environment.
2. Environment: The external system with which the agent interacts.
3. State: A representation of the current situation of the agent within the environment.
4. Action: The choices available to the agent at any given state.
5. Reward: The feedback signal that the agent receives after taking an action, indicating how favorable 

Uma vez que nossa `response` é um outro objeto do tipo `AIMessage`, podemos adicionar ela à lista `messages` e outra `HumanMessage` para gerar uma conversação, tomando em vista a memória do que já foi conversado.

In [7]:
# Adicionando a resposta da IA à messages
messages.append(response)

# Criando um novo user prompt
prompt = HumanMessage(content="Give me some Agent examples.")

# Adicionando o user prompt à messages
messages.append(prompt)

# Requisitando uma resposta do ChatGPT
response = chat(messages)

print(response.content)

Sure! Here are some examples of agents in the context of reinforcement learning:

1. **Autonomous Driving Agent**: An agent that learns to navigate a car through traffic by receiving rewards for safe driving behaviors and penalties for accidents or traffic violations.

2. **Game-Playing Agent**: An agent that learns to play video games by receiving rewards for achieving high scores or completing levels efficiently.

3. **Robotic Arm Agent**: An agent that learns to control a robotic arm to perform specific tasks, such as picking and placing objects, based on feedback from sensors and actuators.

4. **Recommendation System Agent**: An agent that learns user preferences and makes personalized recommendations for products, movies, or content based on feedback and user interactions.

5. **Financial Trading Agent**: An agent that learns to make profitable investment decisions in the stock market by receiving rewards for successful trades and penalties for losses.

These are just a few examp

Legal, temos o nosso bot, porém nada novo por agora. Então, o que podemos ver em seguida é a questão do conhecimento dos LLMs. Apesar de grande parte dos LLMs hoje existentes conseguirem responder uma gama enorme de perguntas diversas com bastante eficácia, nós devemos ter em mente que seu conhecimento não é infinito. O seu conhecimento é limitado pelo seu treinamento, com os dados de treino comprimidos em seus parâmetros internos. Assim, essencialmente, eles aprendem tudo o que sabem durante o treino, e isto implica que eles não podem ter acesso a dados externos por si só.

Podemos ver isso de forma clara quando perguntamos algo recente ao LLM (neste caso, o gpt-3.5-turbo).

In [8]:
# Adicionando a resposta da IA à messages
messages.append(response)

# Criando um novo user prompt
prompt = HumanMessage(content="Do you know de RL4RS framework?")

# Adicionando o user prompt à messages
messages.append(prompt)

# Requisitando uma resposta do ChatGPT
response = chat(messages)

print(response.content)

I'm not familiar with the specific term "RL4RS framework." It's possible that it refers to a specific framework or approach related to reinforcement learning, but without more context or information, I can't provide specific details about it.

If you can provide more details or context about the RL4RS framework, I'd be happy to try to help you understand it better or provide information on a related topic. Feel free to share more information or ask any other questions you may have.


Nesse sentido, como podemos então obter respostas dado um conteúdo atual? E se o LLM me respondesse algo que não é verdade? Ou fazemos um novo ajuste-fino com novos dados (no entanto, muitas vezes isso pode ser inviável dependendo do tipo de problema ou caso de uso que você quer solucionar) ou implementamos justamente o RAG, que insere contexto externo para o LLM ter conteúdo suficiente para dar uma resposta. 

Antes de implementarmos o conceito de RAG a fim de inserir este tipo de conhecimento externo, vamos primeiramente construir a base deste conhecimento.


#### Building the Knowledge Base

Como apresentado no _tópico 1_, aqui estaremos basicamente implementando a seção de `data ingestation`. Neste caso espeífico, estarei utilizando o Facebook AI Similarity Search (`FAISS`) como nosso Vector Database, que basicamente é uma biblioteca para pesquisa eficiente de similaridade e agrupamento de vetores densos e possui uma usabilidade fácil. Eu irei alimentá-lo com o paper _Deep Reinforcement Learning based Recommendation with Explicit User-Item Interactions Modeling_.

Para isso, vamos primeiro utilizar um objeto do tipo `Document Loader` disponibilizado pelo LangChain. Um objeto `Document Loader` basicamente é usado para carregar dados de uma fonte como documentos, em que um documento é um pedaço de texto com metadados associados. Eles podem carregar desde dados de arquivos .txt simples a carregar transcrições de vídeos do Youtube. 

Neste caso, como um paper é um PDF, estaremos utilizando um objeto `Document Loader` que lê PDF's, sendo mais específico, vamos utilizar o `PyPDFLoader`.

In [9]:
loader = PyPDFLoader("DRR-framework.pdf")
drr_framework = loader.load()

In [10]:
drr_framework

[Document(page_content='Deep Reinforcement Learning based\nRecommendation with Explicit User-Item\nInteractions Modeling\nFeng Liu∗, Ruiming Tang†, Xutao Li∗, Weinan Zhang‡Yunming Ye∗, Haokun Chen‡, Huifeng Guo†and Yuzhou Zhang†\n∗Shenzhen Key Laboratory of Internet Information Collaboration\nShenzhen Graduate School, Harbin Institute of Technology, Shenzhen, 518055, China\nEmail: fengliu@stu.hit.edu.cn, lixutao@hit.edu.cn, yeyunming@hit.edu.cn\n†Noah’s Ark Lab, Huawei, China\nEmail: tangruiming, huifeng.guo, zhangyuzhou3@huawei.com\n‡Shanghai Jiao Tong University, Shanghai, China\nEmail: wnzhang@sjtu.edu.cn, chenhaokun@sjtu.edu.cn\nAbstract —Recommendation is crucial in both academia and\nindustry, and various techniques are proposed such as content-\nbased collaborative ﬁltering, matrix factorization, logistic re-\ngression, factorization machines, neural networks and multi-\narmed bandits. However, most of the previous studies suffer\nfrom two limitations: (1) considering the recomm

O que basicamente aconteceu foi: nós carregamos o artigo, lemos cada página e criamos objetos `Document` com as informações e metadados de cada página do artigo.

Se você dar uma olhada no artigo, poderá ver que ele tem exatamente 11 páginas totais. Podemos ver isso também pelo tamanho da lista de documentos `drr_framework`, uma vez que cada objeto foi criado a partir de cada página.

In [12]:
len(drr_framework)

11

Para analisarmos o conteúdo existente em cada página, basta extrair o atributo `page_content` do objeto.

In [14]:
drr_framework[0].page_content

'Deep Reinforcement Learning based\nRecommendation with Explicit User-Item\nInteractions Modeling\nFeng Liu∗, Ruiming Tang†, Xutao Li∗, Weinan Zhang‡Yunming Ye∗, Haokun Chen‡, Huifeng Guo†and Yuzhou Zhang†\n∗Shenzhen Key Laboratory of Internet Information Collaboration\nShenzhen Graduate School, Harbin Institute of Technology, Shenzhen, 518055, China\nEmail: fengliu@stu.hit.edu.cn, lixutao@hit.edu.cn, yeyunming@hit.edu.cn\n†Noah’s Ark Lab, Huawei, China\nEmail: tangruiming, huifeng.guo, zhangyuzhou3@huawei.com\n‡Shanghai Jiao Tong University, Shanghai, China\nEmail: wnzhang@sjtu.edu.cn, chenhaokun@sjtu.edu.cn\nAbstract —Recommendation is crucial in both academia and\nindustry, and various techniques are proposed such as content-\nbased collaborative ﬁltering, matrix factorization, logistic re-\ngression, factorization machines, neural networks and multi-\narmed bandits. However, most of the previous studies suffer\nfrom two limitations: (1) considering the recommendation as\na static p

Meu objetivo agora é dividir todo o conteúdo do artigo em pedaços, isto é, `chunks`. Para isso, vamos utilizar um dos `Text Splliters` do LangChain, sendo mais específico o `RecursiveCharacterTextSplitter`, que é um divisor de textos por caractere.

In [60]:
# Primeiro tomamos todo o conteúdo presente no artigo
content_drr_framework = ''.join([doc.page_content for doc in drr_framework])

# Configuramos o text_splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=100,
    length_function=len,
)

# Criamos os chunks
texts = text_splitter.create_documents([content_drr_framework])

In [61]:
texts

[Document(page_content='Deep Reinforcement Learning based\nRecommendation with Explicit User-Item\nInteractions Modeling\nFeng Liu∗, Ruiming Tang†, Xutao Li∗, Weinan Zhang‡Yunming Ye∗, Haokun Chen‡, Huifeng Guo†and Yuzhou Zhang†\n∗Shenzhen Key Laboratory of Internet Information Collaboration\nShenzhen Graduate School, Harbin Institute of Technology, Shenzhen, 518055, China\nEmail: fengliu@stu.hit.edu.cn, lixutao@hit.edu.cn, yeyunming@hit.edu.cn\n†Noah’s Ark Lab, Huawei, China\nEmail: tangruiming, huifeng.guo, zhangyuzhou3@huawei.com\n‡Shanghai Jiao Tong University, Shanghai, China\nEmail: wnzhang@sjtu.edu.cn, chenhaokun@sjtu.edu.cn\nAbstract —Recommendation is crucial in both academia and\nindustry, and various techniques are proposed such as content-\nbased collaborative ﬁltering, matrix factorization, logistic re-\ngression, factorization machines, neural networks and multi-\narmed bandits. However, most of the previous studies suffer\nfrom two limitations: (1) considering the recomm

Note que agora temos os nossos `chunks`, em uma quantidade maior.

In [62]:
len(texts)

19

Dado os nossos chunks, vamos incorporá-los em vetores e alimentar o FAISS (um Vector Database) com essas incorporações. Nesse sentido, estarei usando o modelo de embedding da OpenAI, o `text-embedding-3-large`. Podemos acessá-lo via LangChain da seguinte forma:

In [21]:
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, 
                              model="text-embedding-3-large")

Agora, nós alimentamos o FAISS a partir dos embeddings dos documentos dos chunks.

In [63]:
vec_database = FAISS.from_documents(texts, embeddings)

In [64]:
vec_database

Vamos dar uma olhada num dos textos que ele retorna quando consultamos: `As for the reward function, what range were the ratings normalized to?`. Por padrão, utilizando o método abaixo, ele retorna os **4** textos/documentos mais similares. Irei selecionar o que ele tomou como mais similar.

In [72]:
docs = vec_database.similarity_search("As for the reward function, what range were the ratings normalized to?")
docs[0].page_content

'RL based models on recommender systems. Therefore, we\nconduct online evaluation with an environment simulator. In\nthis paper, we pretrain a PMF [37] model as the environmentsimulator, i.e., to predict an item’s feedback that the user\nnever rates before. The online evaluation procedure follows\nAlgorithm 1, i.e., the parameters continuously update dur-\ning the online evaluation stage. Its major difference from\nAlgorithm 1 is that the feedback of a recommended item\nis observed by the environment simulator. Moreover, before\neach recommendation session starting in the simulated online\nevaluation, we reset the parameters back to θandωwhich is\nthe policy learned in the training stage for a fair comparison.\nV. E XPERIMENT\nA. Datasets and Evaluation Metrics\nWe adopt the following publicly available datasets from the\nreal world to conduct the experiments:\n•MovieLens (100k)4. A benchmark dataset comprises\nof 0.1 million ratings from users to the recommended\nmovies on MovieLens w

#### Retrieval Augmented Generation

Antes de implementarmos de fato a arquitetura do `Naive RAG`, vamos ver como o nosso chat se sai sem contexto externo.

In [73]:
# Criando a lista de messages
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi Chat, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="Do you know about Deep Reinforcement Learning Based Recommendation With Explicit User-Item Interactions Modeling paper?")
]

In [74]:
response = chat(messages)
print(response.content)

Yes, I am familiar with the paper on Deep Reinforcement Learning Based Recommendation with Explicit User-Item Interactions Modeling. It proposes a method that combines deep reinforcement learning with explicit modeling of user-item interactions to improve recommendation systems. The approach aims to better capture user preferences and provide more accurate recommendations. Is there anything specific you would like to know or discuss about this paper?


In [75]:
# Adicionando a resposta da IA à messages
messages.append(response)

# Criando um novo user prompt
prompt = HumanMessage(content="Tell me about DRR-ave. What is that?")

# Adicionando o user prompt à messages
messages.append(prompt)

# Requisitando uma resposta do ChatGPT
response = chat(messages)

print(response.content)

DRR-ave stands for Deep Reinforcement Learning Based Recommendation with Explicit User-Item Interactions Modeling (DRR-ave). It is a method proposed in a research paper that combines deep reinforcement learning with explicit modeling of user-item interactions to enhance recommendation systems. By incorporating user feedback and interactions with items, DRR-ave aims to provide more personalized and accurate recommendations to users. This approach leverages reinforcement learning techniques to learn optimal recommendation policies based on user preferences and behaviors.


Como eu já estou um pouco mais famialiarizado com este artigo, posso afirmar que essa não é uma resposta certa. Na verdade, aqui está um exemplo de **alucinação**. Vamos fazer mais uma pergunta.

In [76]:
# Adicionando a resposta da IA à messages
messages.append(response)

# Criando um novo user prompt
prompt = HumanMessage(content="How was the state representation module defined in the article?")

# Adicionando o user prompt à messages
messages.append(prompt)

# Requisitando uma resposta do ChatGPT
response = chat(messages)

print(response.content)

In the paper on Deep Reinforcement Learning Based Recommendation with Explicit User-Item Interactions Modeling, the state representation module was defined as a component responsible for encoding the user's historical interactions and preferences into a meaningful representation that can be used by the recommendation system. The state representation module typically takes as input the user's interaction history with items, such as clicks, purchases, ratings, etc., and processes this information to create a condensed and informative representation of the user's current state.

The specific design and implementation of the state representation module may vary depending on the recommendation system and the underlying data. Common techniques used in the state representation module include neural networks, embeddings, and other machine learning models that can capture complex relationships between users and items. The goal of the state representation module is to transform raw user-item int

Aqui está um exemplo de mais uma resposta sem muito sentido. No entanto, esse não é o fim. 

Tendo a nossa base de conhecimento pronta, podemos agora de fato implementar a arquitetura do `Naive RAG` e ver o poder da geração aumentada por recuperação. Quando fizermos uma consulta, tomaremos os **4** textos mais similares em relação a nossa consulta (esse valor pode ser redefinido setando algum outro valor no parâmetro k, e vale lembrar que 4 é o valor padrão) e os passamos como contexto ao LLM para ele ser capaz de responder com fidelidade (além de passar a própria consulta).


In [77]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vec_database.similarity_search(query)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Answer the query using the contexts below.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [78]:
# Setando novamente a lista de messages e gerando uma nova resposta
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi Chat, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content=augment_prompt("Do you know about Deep Reinforcement Learning Based Recommendation With Explicit User-Item Interactions Modeling paper?"))
]

response = chat(messages)
print(response.content)

Yes, I am familiar with the "Deep Reinforcement Learning Based Recommendation With Explicit User-Item Interactions Modeling" paper. It proposes a novel recommendation framework called DRR, which is based on deep reinforcement learning. The framework treats recommendation as a sequential decision-making procedure and uses an "Actor-Critic" reinforcement learning scheme to model interactions between users and recommender systems. It aims to address the limitations of previous studies by considering dynamic adaptation and long-term rewards in the recommendation process. The paper conducts experiments on real-world datasets and shows that the DRR method outperforms existing techniques.


Wow, essa é uma resposta bem mais concisa! Vamos continuar dando uma olhada com as mesmas perguntas anteriores.

In [79]:
# Adicionando a resposta da IA à messages
messages.append(response)

# Criando um novo user prompt
prompt = HumanMessage(content=augment_prompt("Tell me about DRR-ave. What is that?"))

# Adicionando o user prompt à messages
messages.append(prompt)

# Requisitando uma resposta do ChatGPT
response = chat(messages)

print(response.content)

DRR-ave is a structure proposed in the DRR framework for modeling the state representation module. In DRR-ave, the interactions between users and items are explicitly modeled. The structure involves transforming the embeddings of items using a weighted average pooling layer, which is denoted as g(ia). These transformed vectors are then used to model the interactions with the input user. The state representation in DRR-ave is a concatenated vector that includes the user embedding, the interaction vector, and the average pooling result of items.

The key idea behind DRR-ave is to capture the dynamic interactions between users and items while eliminating the position effects in the long-term sequence of items. By incorporating the user-item interactions and leveraging the average pooling technique, DRR-ave aims to provide a more effective and personalized recommendation approach compared to other structures like DRR-p and DRR-u. Experimental results have shown that DRR-ave outperforms oth

Ótimo! Você pode comparar a diferença entre as respostas e concluir que não houve mais alucinação do modelo. Vamos mais uma vez.

In [80]:
# Adicionando a resposta da IA à messages
messages.append(response)

# Criando um novo user prompt
prompt = HumanMessage(content=augment_prompt("How was the state representation module defined in the article?"))

# Adicionando o user prompt à messages
messages.append(prompt)

# Requisitando uma resposta do ChatGPT
response = chat(messages)

print(response.content)

The state representation module in the article was defined by explicitly modeling the interactions between users and items. Three different structures were proposed within the state representation module in the DRR framework:

1. DRR-p: This structure utilizes a product-based neural network to capture the pairwise local dependency between items. It clones the representations of the n items and computes the pairwise interactions between them using an element-wise product operator. Each item's importance is indicated by a scalar weight, and the interactions between items are represented by k-dimensional vectors. The state representation in DRR-p is a concatenation of the cloned vectors and the pairwise interaction vectors.

2. DRR-u: While DRR-p focuses on modeling the pairwise local dependency between items, DRR-u incorporates user-item interactions as well. This structure includes the user embedding and considers the pairwise interactions between users and items in addition to the loca

Incrível! Obtemos respostas corretas e mais concisas e sem a necessidade de um novo ajuste-fino. Agora, terei uma nova ferramenta auxiliar aos meus estudos acerca do artigo e sobre a implementação do mesmo. Maravilhoso, não? 

Agora, vamos dar uma olhada na implementação com o **Llama Index**.

####  2.2.2. Com Llama Index